<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-settings" data-toc-modified-id="Imports-and-settings-1">Imports and settings</a></span></li><li><span><a href="#Table" data-toc-modified-id="Table-2">Table</a></span><ul class="toc-item"><li><span><a href="#Example" data-toc-modified-id="Example-2.1">Example</a></span></li><li><span><a href="#Table-for-list" data-toc-modified-id="Table-for-list-2.2">Table for list</a></span></li><li><span><a href="#Updating-table" data-toc-modified-id="Updating-table-2.3">Updating table</a></span></li></ul></li><li><span><a href="#Changing-bokeh-widgets" data-toc-modified-id="Changing-bokeh-widgets-3">Changing bokeh widgets</a></span><ul class="toc-item"><li><span><a href="#Example" data-toc-modified-id="Example-3.1">Example</a></span></li><li><span><a href="#2D,3D-complete" data-toc-modified-id="2D,3D-complete-3.2">2D,3D complete</a></span></li></ul></li><li><span><a href="#Processed-data" data-toc-modified-id="Processed-data-4">Processed data</a></span></li></ul></div>

# Imports and settings

In [17]:
from bokeh.io import *
from bokeh.layouts import *
from bokeh.plotting import *
from bokeh.models.renderers import *
from bokeh.palettes import *
from bokeh.models.widgets import *
from bokeh.models import *
from bokeh import events

from NearFieldOptics.Materials import *
from NearFieldOptics import TipModels as T
import copy
import numpy as np
from matplotlib import pyplot as plt

output_notebook()
# output_file("new GUI layout.html")

Loading BokehJS ...

In [2]:
from datetime import date
from random import randint

# Table

## Example

In [3]:
data = dict(
        material=[date(2014, 3, i+1) for i in range(10)],
        thickness=[randint(0, 100) for i in range(10)],
    )
source = ColumnDataSource(data)

columns = [
        TableColumn(field="material", title="Material", formatter=DateFormatter()),
        TableColumn(field="thickness", title="thickness (nm)"),
    ]
data_table = DataTable(source=source, columns=columns, width=400, height=280)

add = Button(label="Add!", button_type="success")

show(WidgetBox(data_table,add))

In [4]:
data = dict(
        material=[date(2014, 3, i+1) for i in range(10)],
        thickness=[randint(0, 100) for i in range(10)],
    )

In [5]:
data

{'material': [datetime.date(2014, 3, 1),
  datetime.date(2014, 3, 2),
  datetime.date(2014, 3, 3),
  datetime.date(2014, 3, 4),
  datetime.date(2014, 3, 5),
  datetime.date(2014, 3, 6),
  datetime.date(2014, 3, 7),
  datetime.date(2014, 3, 8),
  datetime.date(2014, 3, 9),
  datetime.date(2014, 3, 10)],
 'thickness': [87, 97, 40, 91, 69, 40, 26, 30, 26, 5]}

## Table for list

In [6]:
data = {}
data["material"] = ['Si','SiO2']
data["thickness"] = [30,15]
source = ColumnDataSource(data)

columns = [
        TableColumn(field="material", title="Material"),
        TableColumn(field="thickness", title="thickness (nm)"),
    ]
data_table = DataTable(source=source, columns=columns, width=400, height=280)

show(data_table)

## Updating table

In [11]:
# Starting bokeh server from here
def modify_doc(doc):
    import subprocess
    import time
    args = ['python', '-m', 'bokeh', 'serve']
    p = subprocess.Popen(args) 
    time.sleep(1) # time to wait might be longer on other PC

    def create_cds():
        '''Create two different column data sources'''

        data1 = dict(dates=[date(2014, 3, i+1) for i in range(10)],
                     downloads=[randint(0, 100) for i in range(10)])
        source1 = ColumnDataSource(data1)

        data2 = dict(dates=[date(2017, 3, i+1) for i in range(15)],
                     downloads=[randint(100, 200) for i in range(15)])
        source2 = ColumnDataSource(data2)

        return source1, source2

    # defining initial CDS and initial table

    source1, source2 = create_cds()

    source = source1
    columns = [TableColumn(field="dates", title="Date", formatter=DateFormatter()),
               TableColumn(field="downloads", title="Downloads")]
    data_table = DataTable(source=source, columns=columns, width=400, height=280)

    # callback for RadioButtonGroup
    def select_data(attr, old, new):
        
        source1, source2 = create_cds()

        if new == 0: 
            source.data = source1.data
        else: 
            source.data = source2.data

        global data_table
        data_table.update()

    button = RadioButtonGroup(labels=["Option 0", "Option 1"], active=0)
    button.on_change('active', select_data)

    mylayout = layout([[button], [data_table]])
    doc.add_root(mylayout)
    
show(modify_doc)

# Changing bokeh widgets

## Example

In [8]:
def modify_doc(doc):

    start_button = Button(label="Start the experiment") 
    stop_button = Button(label="Stop", disabled=True)
    m1_text = TextInput(title="mass of puck1 =", value="570")
    spark_timer = Slider(value=0, start=0, end=100, step=1, title="Title")

    button_list = ["Start_Button","Stop_Button","Mass_of_m1","Spark"]

    button_call = Select(title="Button call", options=sorted(button_list), value="Start_Button",width=200)

    select_code = """
    to_show = cb_obj.value;
    options = cb_obj.options

    for (i=0;i<options.length;i++) {
        box = document.getElementsByClassName(options[i])[0];
        if (!box.className.includes("hidden")) {box.className+=" hidden";}
        if (box.className.includes(to_show)) {box.className=box.className.replace(" hidden","");}
    }
    """

    button_call.callback = CustomJS(code=select_code)

    box_1 = widgetbox(start_button,css_classes = ['Start_Button'])
    box_2 = widgetbox(stop_button,css_classes = ['Stop_Button','hidden'])
    box_3 = widgetbox(m1_text,css_classes = ['Mass_of_m1','hidden'])
    box_4 = widgetbox(spark_timer,css_classes = ['Spark','hidden'])


    grid = gridplot([[button_call],[box_1,box_2,box_3,box_4]],toolbar_location=None)
    doc.add_root(grid)
    
show(modify_doc)

## 2D,3D complete

In [21]:
def header():
    text = Div(text="<h1><b>The Lightning Rod Model GUI</b></h1>",width=400, height=30)
    title = WidgetBox(text, width=400)

    callback1 = CustomJS(code="alert('Don't drag away the logo!')")
    tap = TapTool(callback=callback1)
    cu = figure(x_range=(0,6), y_range=(-3.5,0.2),width=250,height=60,toolbar_location=None, tools=[tap])
    cu.outline_line_color = "white"
    cu.image_url(url=['Columbia logo.png'], x=0, y=0.2,w=6, h=3.5)
    cu.xgrid.grid_line_color = None
    cu.ygrid.grid_line_color = None
    cu.axis.visible = False

    callback1 = CustomJS(code="alert('Don't drag away the logo!')")
    tap = TapTool(callback=callback1)
    basov = figure(x_range=(0,6), y_range=(-3.5,0.2),width=150,height=60,toolbar_location=None, tools=[tap])
    basov.outline_line_color = "white"
    basov.image_url(url=['basov lab logo.png'], x=0, y=0.2,w=6, h=3.5)
    basov.xgrid.grid_line_color = None
    basov.ygrid.grid_line_color = None
    basov.axis.visible = False
    return [title,cu,basov]

def space():
    empty = Div(text="",width=400, height=1)
    space = WidgetBox(empty, width=400)
    return [empty]

In [28]:
def modify_doc(doc):
    
    global layerList,computerLayerList,tabulatedData
    tabulatedData = {}
    layerList = []
    computerLayerList = []
    
    material_list = ["SiO2_300nm","SingleLayerGraphene","BN_Caldwell","Si_80K","Si_150K","Si_300K","FePO4","TaS2_CCDW","TaS2_NCCDW","Silicon","Air"]
    material_list_without_surface = copy.copy(material_list)
    material_list_without_surface.remove("SingleLayerGraphene")
    
    entranceMaterialMenu = Select(title="Entrance material:", value="Air", options=sorted(material_list))
    exitMaterialMenu = Select(title="Exit material:", value="Air", options=sorted(material_list))
    materialMenu = Select(title="Material", options=sorted(material_list), value="SiO2_300nm",width=200)
    
    thicknessText = TextInput(title="Thickness (nm):", value="0")
    muText = TextInput(title="Chemical potential (cm-1):", value="1000")
    gammaText = TextInput(title="Scattering rate ():", value="0.1")
    
    add = Button(label="Add", button_type="primary")
    assemble = Button(label="ASSEMBLE!", button_type="success")
    clear = Button(label="CLEAR ALL", button_type="warning")
    
    tabulatedData["material"] = []
    tabulatedData["thickness"] = []
    tabulatedData["mu"] = []
    tabulatedData["gamma"] = []
    
    source1 = ColumnDataSource(tabulatedData)
    source = source1
    columns = [
            TableColumn(field="material", title="Material"),
            TableColumn(field="thickness", title="thickness (nm)"),
            TableColumn(field="mu", title="chemical potential (cm-1)"),
            TableColumn(field="gamma", title="scatting rate ()"),
        ]
    table = DataTable(source=source, columns=columns, width=700, height=280)
    
    select_code = """
    to_show = cb_obj.value;
    options = cb_obj.options

    for (i=0;i<options.length;i++) {
        box = document.getElementsByClassName(options[i])[0];
        if (!box.className.includes("hidden")) {box.className+=" hidden";}
        if (box.className.includes(to_show)) {box.className=box.className.replace(" hidden","");}
    }
    """
    
    materialMenu.callback = CustomJS(code=select_code)
    
    #Add a layer
    def update_parameters():
        
        if materialMenu.value=="SingleLayerGraphene":
            layerList.append(("SingleLayerGraphene",muText.value,gammaText.value))
            tabulatedData["material"].append("SingleLayerGraphene")
            tabulatedData["thickness"].append("/")
            tabulatedData["mu"].append(muText.value)
            tabulatedData["gamma"].append(gammaText.value)
            
        else:
            layerList.append((materialMenu.value,float(thicknessText.value)))
            tabulatedData["material"].append(materialMenu.value)
            tabulatedData["thickness"].append(thicknessText.value)
            tabulatedData["mu"].append("/")
            tabulatedData["gamma"].append("/")
        
        columns = [
                    TableColumn(field="material", title="Material"),
                    TableColumn(field="thickness", title="thickness (nm)"),
                    TableColumn(field="mu", title="chemical potential (cm-1)"),
                    TableColumn(field="gamma", title="scatting rate ()")
                  ]
        
        source1 = ColumnDataSource(tabulatedData)
        source.data = source1.data
        global table
        table.update()
    
    #Assemble a LayeredMedia
    def assemble_layered_media():
        global entranceMaterial,exitMaterial
        entranceMaterial = _get_computer_material_(entranceMaterialMenu.value)
        exitMaterial = _get_computer_material_(exitMaterialMenu.value)
        
        for layer in layerList:
            materialString = layer[0]
            if materialString=="SingleLayerGraphene":
                surface = SingleLayerGraphene(chemical_potential=layer[1],gamma=layer[2])
                computerLayerList.append(surface)
            else:
                thickness = layer[1]*1e-7
                if materialString=='Si_80K' or materialString=='Si_150K' or materialString=='Si_300K':
                    computerLayer = TabulatedMaterialFromFile(materialString+'.csv')
                else:
                    computerLayer = _get_computer_material_(materialString)
                computerLayerList.append((computerLayer,thickness))
    def _get_computer_material_(matString):
        
        if matString == "SiO2_300nm":
            return SiO2_300nm
        elif matString == "BN_Caldwell":
            return BN_Caldwell
        elif matString == "FePO4":
            return FePO4
        elif matString == "TaS2_CCDW":
            return TaS2_CCDW
        elif matString == "TaS2_NCCDW":
            return TaS2_NCCDW
        elif matString == "Si":
            return Si
        elif matString == "Air":
            return Air
        else:
            Logger.raiseException('Material not recognized.', exception=valueError)
            return None
            
    #Clear all inputs and stored values
    def clear_all():
        layerList.clear()
        computerLayerList.clear()
        layerMedia = None
        
        tabulatedData["material"] = []
        tabulatedData["thickness"] = []
        columns = [
            TableColumn(field="material", title="Material"),
            TableColumn(field="thickness", title="thickness (nm)"),
        ]
        source1 = ColumnDataSource(tabulatedData)
        source.data = source1.data
        global table
        table.update()
    
    add.on_click(update_parameters)
    assemble.on_click(assemble_layered_media)
    clear.on_click(clear_all)
    
    box0 = widgetbox(entranceMaterialMenu,exitMaterialMenu)
    box01 = widgetbox(materialMenu)
    box1 = widgetbox(muText,gammaText,css_classes = ['SingleLayerGraphene','hidden'])
    box2 = widgetbox(thicknessText,css_classes = material_list_without_surface)
    box3 = widgetbox(add,assemble,clear)
    
    l = layout([box0,box01,box1,box2,box3],table)
    doc.add_root(l)
    
show(modify_doc)

# Processed data

In [13]:
tabulatedData

{'material': ['SiO2_300nm', 'SiO2_300nm'],
 'thickness': ['30', '30'],
 'mu': ['/', '/'],
 'gamma': ['/', '/']}

In [31]:
layerList

[('SiO2_300nm', 30.0)]

In [16]:
computerLayerList

[(<NearFieldOptics.Materials.material_types.IsotropicMaterial at 0x10dcf4e80>,
  3e-06),
 (<NearFieldOptics.Materials.material_types.IsotropicMaterial at 0x10dcf4e80>,
  3e-06)]

In [ ]:
LM = LayeredMediaTM(layerArrayGUIInput = computerLayerList, entrance=entranceMaterial,exit=exitMaterial)